In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from itertools import chain
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Evaluating Decision Tree Model
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    f1_score,
    recall_score,
    precision_score,
    confusion_matrix,
    classification_report,
)
from sklearn.model_selection import (
    RepeatedStratifiedKFold,
    GridSearchCV,
    cross_val_score,
)
import sklearn
from sklearn.utils import class_weight

# Engaging in Data Preprocessing
from sklearn.model_selection import train_test_split as tts

from sklearn.feature_selection import RFE

# # Visualizing the Decision Trees generated
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from sklearn.tree import export_graphviz

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [3]:
df = pd.read_csv("../14_profiles/reduced_dim_14.csv")
df

,profile_standardized,.NET,.NET Core,API,API Design,APIs,ASP.NET,ASP.NET MVC,AWS,AWS DynamoDB,...,Xamarin,Xamarin Forms,Xcode,Zend Framework,iOS,iOS Operating System,iOS SDK,iPad,jQuery,js
0,fullstack developer,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,javascript developer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,machine learning engineer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,machine learning engineer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,android developer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,fullstack developer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
767,machine learning engineer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
768,ios developer,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
769,javascript developer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

In [4]:
# Assign data to X & y
X = df.iloc[:, 1:]
y = df.profile_standardized

<IPython.core.display.Javascript object>

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=0)

<IPython.core.display.Javascript object>

In [6]:
# Fit the DecisionTreeClassifier model
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

<IPython.core.display.Javascript object>

### Establish Benchmark Model Performance

In [7]:
# Make predictions using the testing data
y_pred = model.predict(X_test)
print(
    "f1_micro score of decision tree classifier on test set: {:.3f}".format(
        np.mean(cross_val_score(model, X_test, y_test, cv=10, scoring="f1_micro"))
    )
)

f1_micro score of decision tree classifier on test set: 0.410


<IPython.core.display.Javascript object>

### Conduct RFE

In [8]:
# Run RFE
rfe = RFE(model)
rfe = rfe.fit(X, y)

<IPython.core.display.Javascript object>

In [9]:
# returns bool arr of True for features_selected
mask = rfe.support_
# Rank of 1 indicates features selected
print(rfe.ranking_)

[153 152 151 150 149 147 146   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1  20   1   1  23  30  12
  11  29   1   1  22   1  31   1   1  35   1   1   1   1  68  59   1   1
  42   1   1   1   1   1   1  44   1  41   1   5  28   1   1   1  81  37
   1  99  16  90 129   1  67   1   1 139  91   1 104  55   1   1  63  82
   1  79  73   1 141  58   1   1  71  92  85   1  49  61 130   1  77   1
   1   1   1   1   1   1  27  94 135 121 137   1   1   7 116 142 118  78
 131   1 126 132   1   1   1 136 120   1  17   9   1   1   1   1   1  51
 110   1   1   1 111   1 143   1 154 145 105   1 144   1   1   1 128  32
   1  74  43   1   1   1  72  39  60   1   1  14   1  96  33   1  64   1
   1   1  18   1   1   1  21   1  24   1  19   1  13   3  36   2   1   1
   1  45   1  88   1   1  40   1  69   1   1   1  46   4   1  47   1   1
  56  76  62 101   1  54  75   1  57  80  95  25  93   1   1 113   1   1
 100  98  97  66 140   1  89 138  83  38  34   1  5

<IPython.core.display.Javascript object>

In [10]:
# RFE output = 152 features
selection = rfe.get_support(1)
print(len(selection))

152


<IPython.core.display.Javascript object>

In [11]:
# Subset X by selected features
X = X[X.columns[selection]]

<IPython.core.display.Javascript object>

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=0)

<IPython.core.display.Javascript object>

In [13]:
# Fit the DecisionTreeClassifier model
rfe_model = DecisionTreeClassifier(random_state=0)
rfe_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

<IPython.core.display.Javascript object>

In [14]:
# Generate Feature Importance DataFrame of Importances > .01
feat_imp = pd.DataFrame(zip(X.columns, rfe_model.feature_importances_)).rename(
    columns={0: "feature", 1: "importance"}
)
mask = feat_imp.importance > 0.01
feat_imp = feat_imp[mask]

<IPython.core.display.Javascript object>

<Figure size 576x1728 with 0 Axes>

<IPython.core.display.Javascript object>

In [15]:
features = feat_imp.feature
y_vals = np.sort(feat_imp.importance)

# plot
fig, ax = plt.subplots()
width = 0.8
ind = np.arange(len(y_vals))

ax.barh(ind, y_vals, width)
ax.set_yticks(ind + width / 10)
ax.set_yticklabels(features, fontsize=12, rotation=0, minor=False)

plt.title("Feature importance in Decision Tree Classifier", fontsize=18)
plt.xlabel("Relative importance")
plt.ylabel("Feature")
plt.figure(figsize=(8, 24))
plt.savefig("Feature importance in Decision Tree Classifier.pdf")
fig.set_size_inches(8, 14, forward=True)

### Compare Model Performance with Class Weights, Hypertuned Parameters and Optimized Number of Features with RFE

In [16]:
# Make predictions using the testing data
y_pred = rfe_model.predict(X_test)
print(
    "f1_micro score of decision tree classifier on test set: {:.3f}".format(
        np.mean(cross_val_score(rfe_model, X_test, y_test, cv=10, scoring="f1_micro"))
    )
)

f1_micro score of decision tree classifier on test set: 0.445


<IPython.core.display.Javascript object>

### Hypertune the parameters using GridSearchCV

In [17]:
# Set Grid & CV Values
grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": range(10, 20),
    "min_samples_split": range(5, 15),
    "min_samples_leaf": range(5, 15),
}
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)

<IPython.core.display.Javascript object>

In [18]:
# Conduct Grid Search
grid_search = GridSearchCV(
    estimator=model,
    param_grid=grid,
    n_jobs=-1,
    cv=cv,
    scoring="accuracy",
    error_score=0,
)

grid_result = grid_search.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.434068 using {'criterion': 'gini', 'max_depth': 19, 'min_samples_leaf': 5, 'min_samples_split': 14}


<IPython.core.display.Javascript object>

In [19]:
hypertuned_model = grid_result.best_estimator_

<IPython.core.display.Javascript object>

### Compare Model performance with Hypertuned Parameters

In [20]:
# Make predictions using the testing data
y_pred = hypertuned_model.predict(X_test)
print(
    "f1_micro score of decision tree classifier on test set: {:.3f}".format(
        np.mean(
            cross_val_score(hypertuned_model, X_test, y_test, cv=10, scoring="f1_micro")
        )
    )
)

f1_micro score of decision tree classifier on test set: 0.371


<IPython.core.display.Javascript object>

### Calculate Class Weights

In [21]:
# Count of each type of class
labels_dict = df.profile_standardized.value_counts().to_dict()
labels_dict

{'javascript developer': 138,
 'quality assurance': 86,
 'frontend developer': 73,
 'fullstack developer': 67,
 'android developer': 53,
 'machine learning engineer': 53,
 'python developer': 48,
 'php developer': 47,
 'ios developer': 47,
 'java developer': 45,
 'web design': 43,
 'ruby on rails developer': 24,
 'c# developer': 24,
 'data architecture developer': 23}

<IPython.core.display.Javascript object>

In [22]:
# Calculating class weight
weights = df.profile_standardized.value_counts() / len(df)
# DecisionTreeClassifier Object takes class weights as dict
weights = weights.to_dict()
for key in weights:
    weights[key] *= 100

<IPython.core.display.Javascript object>

In [23]:
# Fit DecisionTreeClassifier Model
class_weights_model = DecisionTreeClassifier(
    random_state=0, class_weight=weights, **grid_result.best_params_
)
class_weights_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight={'android developer': 6.8741893644617384,
                                     'c# developer': 3.11284046692607,
                                     'data architecture developer': 2.9831387808041505,
                                     'frontend developer': 9.46822308690013,
                                     'fullstack developer': 8.690012970168612,
                                     'ios developer': 6.09597924773022,
                                     'java developer': 5.836575875486381,
                                     'javascript developer': 17.898832684824903,
                                     'machine learning engineer': 6.8741893644617384,
                                     'php developer': 6.09597924773022,
                                     'python developer': 6.22568093385214,
                                     'quality assurance': 11.154345006485086,
                                     'ruby on rails developer': 3.112

<IPython.core.display.Javascript object>

### Compare Model Performance with Class Weights

The 'f1_micro' metric is used here as it uses the overall TP, FN, FP, TN to calculate F1 directly, and therefore does not favor any class in particular.

In [24]:
# Make predictions using the testing data
y_pred = class_weights_model.predict(X_test)
print(
    "f1_micro score of decision tree classifier on test set: {:.3f}".format(
        np.mean(
            cross_val_score(
                class_weights_model, X_test, y_test, cv=10, scoring="f1_micro"
            )
        )
    )
)

f1_micro score of decision tree classifier on test set: 0.324


<IPython.core.display.Javascript object>

<Figure size 576x576 with 0 Axes>

<IPython.core.display.Javascript object>

In [25]:
fig, ax = plt.subplots()
plot_tree(rfe_model, filled=True)
plt.figure(figsize=(8, 8))
fig.set_size_inches(8, 6, forward=True)